In [ ]:
import { document } from 'jsr:@ry/jupyter-helper';
import pl from 'npm:nodejs-polars';
import * as Plot from 'npm:@observablehq/plot';
let { html, md, display } = Deno.jupyter;


In [ ]:
type Seriesify<T extends Record<string, pl.DataType>> = {
  [key in keyof T]: key extends string ? pl.Series<T[key], key> : pl.Series<T[key]>;
};


In [ ]:
type Ascent = {
  created_at: pl.Datetime;
  ascendable_id: pl.Int64;
  ascendable_name: pl.String;
  ascendable_type: pl.Categorical; //'GymBoulder';
  grading_system: pl.Categorical; //'font';
  grade: pl.String;
  difficulty: pl.String;
  type: pl.Categorical; // 'rp' | 'f';
  sub_type: pl.String;
  tries: pl.Int64;
};

let ascents = pl.readJSON('./data/ascents_230474.jsonl', {
  inferSchemaLength: null,
  format: 'lines',
}) as pl.DataFrame<Seriesify<Ascent>>;

ascents = ascents
  .withColumns(
    ascents.getColumn('created_at').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S %z').cast(pl.Datetime('ms')),
    ascents.getColumn('updated_at').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S %z').cast(pl.Datetime('ms')),
    ascents.getColumn('date').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S %z').cast(pl.Datetime('ms')),
    ascents.getColumn('ascendable_type').cast(pl.Categorical),
    ascents.getColumn('grading_system').cast(pl.Categorical),
  )
  .withColumns(pl.col('date').date.year().alias('year'))
  // remove repeat ascents
  .unique({ subset: 'ascendable_id', keep: 'first' });


ascendable_type,steepness,eight_a_id,eight_a_user_id,parent_name,ascendable_filter,repeats,project,protection,score,eight_a_parent_id,archived_at,eight_a_ascendable_id,comment,difficulty,grade,id,sub_type,tries,vl_user_id,virtual,exposition,ascendable_name,hold_color,features,vl_parent_id,created_at,date,recommended,path,eight_a_logbook,provider,ascendable_height,grading_system,vl_ascendable_id,ascendable_id,type,sits,user_id,height,shade,repeat,safety_issues,rating,updated_at,parent_id,vl_id,style,perceived_hardness,year
GymBoulder,0,null,133113,Klättercentret Solna,null,0,false,null,103,null,null,null,null,4+,vl-8,15137384,,1,286128,false,null,Dark Green,null,null,null,Sun May 05 2024 15:59:45 GMT+0200 (Central European Summer Time),Sun May 05 2024 14:00:00 GMT+0200 (Central European Summer Time),false,null,false,verticallife,null,font,null,514106,f,0,230474,4,null,false,null,null,Sun May 05 2024 15:59:46 GMT+0200 (Central European Summer Time),69,6553407,,0,2024
GymBoulder,0,null,133113,Klättercentret Solna,null,0,false,null,552,null,null,null,null,6B+,vl-19,16004307,,2,286128,false,null,Purple,null,null,null,Wed Oct 16 2024 19:08:48 GMT+0200 (Central European Summer Time),Wed Oct 16 2024 14:00:00 GMT+0200 (Central European Summer Time),false,null,false,verticallife,null,font,null,552254,rp,0,230474,4,null,false,null,null,Wed Oct 16 2024 19:08:48 GMT+0200 (Central European Summer Time),69,7003719,,0,2024
GymBoulder,0,null,133113,Klättercentret Solna,null,0,false,null,502,null,null,null,null,6B,vl-18,15328296,,2,286128,false,null,White,null,null,null,Thu Jun 06 2024 23:18:15 GMT+0200 (Central European Summer Time),Thu Jun 06 2024 14:00:00 GMT+0200 (Central European Summer Time),false,null,false,verticallife,null,font,null,523297,rp,0,230474,4,null,false,null,null,Thu Jun 06 2024 23:18:16 GMT+0200 (Central European Summer Time),69,6653002,,0,2024
GymBoulder,0,null,133113,Klättercentret Solna,null,0,false,null,103,null,null,null,null,3+,vl-5,5361755,,1,286128,false,0,Turquoise,null,0,null,Wed Oct 04 2023 18:56:25 GMT+0200 (Central European Summer Time),Wed Oct 04 2023 14:00:00 GMT+0200 (Central European Summer Time),false,null,false,verticallife,0,font,null,462730,f,0,230474,4,0,false,0,null,Wed Oct 25 2023 01:37:18 GMT+0200 (Central European Summer Time),69,5934670,,0,2023
GymBoulder,0,null,133113,Klättercentret Solna,null,0,false,0,52,null,null,null,null,4+,vl-8,5505656,,1,286128,false,0,Black,null,0,null,Wed Feb 22 2023 18:17:52 GMT+0100 (Central European Standard Time),Wed Feb 22 2023 13:00:00 GMT+0100 (Central European Standard Time),false,null,false,verticallife,0,font,null,413060,f,0,230474,4,0,false,0,null,Wed Oct 25 2023 01:45:34 GMT+0200 (Central European Summer Time),69,5298229,,0,2023
GymBoulder,0,null,133113,Klättercentret Solna,null,0,false,null,550,null,null,null,null,6B+,vl-19,14598928,,5,286128,false,null,Dark Green,null,null,null,Wed Feb 14 2024 20:04:24 GMT+0100 (Central European Standard Time),Wed Feb 14 2024 13:00:00 GMT+0100 (Central European Standard Time),false,null,false,verticallife,null,font,null,493941,rp,0,230474,4,null,false,null,null,Wed Feb 14 2024 20:04:25 GMT+0100 (Central European Standard Time),69,6307315,,0,2024
GymBoulder,0,null,133113,Klättercentret Solna,null,0,false,null,103,null,null,null,null,3+,vl-5,14172331,,1,286128,false,0,Yellow,null,0,null,Sat Dec 02 2023 16:34:20 GMT+0100 (Central European Standard Time),Sat Dec 02 2023 13:00:00 GMT+0100 (Central European Standard Time),false,null,false,verticallife,0,font,null,478968,f,0,230474,4,0,false,0,null,Sat Dec 02 2023 16:34:20 GMT+0100 (Central European Standard Time),69,6108573,,0,2023
GymBoulder,0,null,133113,Klättercentret Solna,null,0,false,null,253,null,null,null,null,5,vl-12,16453325,,1,286128,false,null,Turquoise,null,null,null,Sat Jan 11 2025 14:34:58 GMT+0100 (Central European Standard Time),Sat Jan 11 2025 13:00:00 GMT+0100 (Central European Standard Time),false,null,false,verticallife,null,font,null,567337,f,0,230474,4,null

In [ ]:
type Boulder = {
  id: pl.Int64;
  route_setter: pl.String;
  sector_name: pl.String;
  gym_wall_name: pl.String;
  set_at: pl.Datetime;
  route_card_label: pl.String;
};

let boulders = pl
  .readJSON('./data/gym_boulders.jsonl', {
    inferSchemaLength: null,
    format: 'lines',
  })
  .withColumns(
    pl.col('route_card_label').str.slice(0, 3),
    pl.col('set_at').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S %z').cast(pl.Datetime('ms')),
    pl.col('id').alias('nid'),
  ) as pl.DataFrame<Seriesify<Boulder>>;


In [ ]:
let detailedAscents = ascents
  .join(boulders, {
    leftOn: 'ascendable_id',
    rightOn: 'nid',
  })
  .sort(pl.col('date'));


In [ ]:
let boulderScores = [
  { score: 12, v: 'VB', font: '2' },
  { score: 18, v: 'VB', font: '2+' },
  { score: 24, v: 'VB+', font: '3' },
  { score: 30, v: 'VB+', font: '3+' },
  { score: 36, v: 'V0-', font: '4' },
  { score: 42, v: 'V0', font: '4+' },
  { score: 48, v: 'V1', font: '5' },
  { score: 54, v: 'V2', font: '5+' },
  { score: 60, v: 'V3', font: '6a' },
  { score: 66, v: 'V3', font: '6a+' },
  { score: 72, v: 'V4', font: '6b' },
  { score: 78, v: 'V4', font: '6b+' },
  { score: 84, v: 'V5', font: '6c' },
  { score: 90, v: 'V5', font: '6c+' },
  { score: 96, v: 'V6', font: '7a' },
  { score: 102, v: 'V7', font: '7a+' },
  { score: 108, v: 'V7', font: '7b' },
  { score: 114, v: 'V8', font: '7b+' },
  { score: 120, v: 'V9', font: '7c' },
  { score: 126, v: 'V10', font: '7c+' },
].map((grade) => ({ ...grade, font: grade.font.toUpperCase() }));
let gradeToNumber = boulderScores.reduce((acc, grade) => {
  acc[grade.font] = grade.score;
  return acc;
}, {});

gradeToNumber;


{
  "2": 12,
  "3": 24,
  "4": 36,
  "5": 48,
  "2+": 18,
  "3+": 30,
  "4+": 42,
  "5+": 54,
  "6A": 60,
  "6A+": 66,
  "6B": 72,
  "6B+": 78,
  "6C": 84,
  "6C+": 90,
  "7A": 96,
  "7A+": 102,
  "7B": 108,
  "7B+": 114,
  "7C": 120,
  "7C+": 126
}

In [ ]:
detailedAscents = detailedAscents.withColumns(
  pl.col('difficulty').replace({ old: gradeToNumber }).cast(pl.Float32).alias('numberDifficulty'),
  pl.col('type').replace(['f', 'rp'], ['flash', 'redpoint']),
);


ascendable_type,steepness,eight_a_id,eight_a_user_id,parent_name,ascendable_filter,repeats,project,protection,score,eight_a_parent_id,archived_at,eight_a_ascendable_id,comment,difficulty,grade,id,sub_type,tries,vl_user_id,virtual,exposition,ascendable_name,hold_color,features,vl_parent_id,created_at,date,recommended,path,eight_a_logbook,provider,ascendable_height,grading_system,vl_ascendable_id,ascendable_id,type,sits,user_id,height,shade,repeat,safety_issues,rating,updated_at,parent_id,vl_id,style,perceived_hardness,year,steepness_right,traverse,sector_id,gym_wall_id,gym,stats,share_url,reference_width,difficulty_right,name,grade_right,hold_manufacturer,item_type,notes,color_1,route_setter,id_right,set_at,archived,virtual_right,sector_name,grade_proposals,expiration_date,topo,path_right,line_number,created_at_right,topo_num,route_card_label,gym_wall_name,grading_system_right,color_2,updated_at_right,gallery,two_start_holds,sit_down_start,gym_id,parent_name_right,style_right,display_route_setter,numberDifficulty
GymBoulder,0,null,133113,Klättercentret Solna,null,0,false,0,353,null,null,null,null,5+,vl-14,4050616,,1,286128,false,null,Blue,null,null,null,Sun Mar 20 2022 16:57:12 GMT+0100 (Central European Standard Time),Sun Mar 20 2022 13:00:00 GMT+0100 (Central European Standard Time),false,null,false,verticallife,null,font,null,322901,flash,0,230474,4,null,false,null,null,Wed Oct 25 2023 00:27:09 GMT+0200 (Central European Summer Time),69,4383951,,0,2022,3,false,219,734,[object Object],[object Object],https://zlag.vertical-life.info/en/routes/a498358a22634efc8ce0b93a66b57dac,971,5+,Blue,vl-14,null,gym_boulder,null,#0000ff,N.N.,322901,Wed Feb 23 2022 17:37:42 GMT+0100 (Central European Standard Time),true,false,Bouldering,"vl-1,2,vl-2,2+,vl-3,3-,vl-4,3,vl-5,3+,vl-6,4-,vl-7,4,vl-8,4+,vl-9,5-,vl-12,5,vl-14,5+,vl-16,6A,vl-17,6A+,vl-18,6B,vl-19,6B+,vl-21,6C,vl-22,6C+,vl-23,7A,vl-24,7A+,vl-25,7B,vl-26,7B+,vl-27,7C,vl-28,7C+,vl-29,8A,vl-30,8A+,vl-31,8B,vl-32,8B+,vl-33,8C,vl-34,8C+",null,e6411c2ffc69fdad028b8408b295e45c,"508,187,689,184,847,182,931,514,470,491,515,312,508,186",3,2022-02-23 16:37:56 +0000,41800.3.,B3.,B Sektionen,font,null,2022-03-28 08:23:57 +0000,,null,null,69,"Bouldering, B Sektionen",null,true,54
GymBoulder,0,null,133113,Klättercentret Solna,null,0,false,0,253,null,null,null,null,5,vl-12,4050981,,1,286128,false,null,Turquoise,null,null,null,Sun Mar 20 2022 16:55:49 GMT+0100 (Central European Standard Time),Sun Mar 20 2022 13:00:00 GMT+0100 (Central European Standard Time),false,null,false,verticallife,null,font,null,320041,flash,0,230474,4,null,false,null,null,Wed Oct 25 2023 00:27:09 GMT+0200 (Central European Summer Time),69,4383933,a,0,2022,7,false,219,733,[object Object],[object Object],https://zlag.vertical-life.info/en/routes/bb4f0a2fdc87192acb532cc5b4c82257,971,5,Turquoise,vl-12,null,gym_boulder,null,#87ccc6,N.N.,320041,Mon Feb 14 2022 19:25:18 GMT+0100 (Central European Standard Time),true,false,Bouldering,"vl-1,2,vl-2,2+,vl-3,3-,vl-4,3,vl-5,3+,vl-6,4-,vl-7,4,vl-8,4+,vl-9,5-,vl-12,5,vl-14,5+,vl-16,6A,vl-17,6A+,vl-18,6B,vl-19,6B+,vl-21,6C,vl-22,6C+,vl-23,7A,vl-24,7A+,vl-25,7B,vl-26,7B+,vl-27,7C,vl-28,7C+,vl-29,8A,vl-30,8A+,vl-31,8B,vl-32,8B+,vl-33,8C,vl-34,8C+",null,e56970f2e4e002f62f72f0061adfa987,"291,182,302,300,294,520,382,504,507,502,721,526,720,489,733,183,565,180,291,182",2,2022-02-14 18:25:35 +0000,41797.2.,A2.,A Sektionen,font,null,2022-03-24 08:21:39 +0000,,null,null,69,"Bouldering, A Sektionen",a,true,48
GymBoulder,0,null,133113,Klättercentret Solna,null,0,false,0,103,null,null,null,null,3,vl-4,4634900,,1,286128,false,null,Turquoise,null,null,null,Sun Mar 20 2022 17:03:56 GMT+0100 (Central European Standard Time),Sun Mar 20 2022 13:00:00 GMT+0100 (Central European Standard Time),false,null,false,verticallife,null,font,null,326080,flash,0,230474,4,null,false,null,null,Wed Oct 25 2023 00:58:25 GMT+0200 (Central European Summer Time),69,4383979,,0,2022,7,false,219,733,[object Object],[object Object],

In [ ]:
html`${detailedAscents.getColumn('route_setter').valueCounts().sort('count', true).toHTML()}`;


route_setter,count
Anton Cuevas,253
Silver Breiktopf,181
James Thanawat,140
Benjamin Schlobohm,137
Stina Dorwarth,88
Sebastian Dahlgren,77
N.N.,69
Robert Rundin,44
Sophie Machaczek,20
Isabelle Fugelstad,19


In [ ]:
let baseWidth = 640;
let ascentTypeColorDomain = { domain: ['flash', 'redpoint'], range: ['#ffd800', '#ff5151'] };


In [ ]:
let groupings = [
  {
    name: 'Everything',
    y: 'difficulty',
    fill: 'type',
  },
  {
    name: 'By year',
    y: 'difficulty',
    fy: 'year',
    fill: 'type',
  },
  {
    name: 'By gym',
    y: 'difficulty',
    fx: 'year',
    fy: 'parent_name',
    fill: 'type',
  },
  {
    name: 'By route setter',
    y: 'difficulty',
    fx: 'year',
    fy: 'route_setter',
    fill: 'type',
    sort: { fy: '-x', reduce: 'count' },
  },
];
for (const grouping of groupings) {
  await display(md`## ${grouping.name}`);
  await display(
    Plot.plot({
      grid: true,
      width: baseWidth * 2,
      color: { legend: true, type: 'categorical', ...ascentTypeColorDomain },
      marginRight: 150,
      marks: [
        Plot.frame(),
        Plot.barX(detailedAscents.toRecords(), Plot.groupY({ x: 'count' }, { ...grouping, title: 'count' })),
      ],
      y: { type: 'band', reverse: true },
      document,
    }),
  );
}


## Everything

flash redpoint 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 difficulty 0 10 20 30 40 50 60 70 80 90 100 110 120 Frequency → undefined (94) undefined (81) undefined (49) undefined (91) undefined (74) undefined (104) undefined (111) undefined (60) undefined (18) undefined (62) undefined (51) undefined (3) undefined (7) undefined (1) undefined (1) undefined (6) undefined (2) undefined (17) undefined (5) undefined (34) undefined (50) undefined (56) undefined (23) undefined (28) undefined (15) undefined (4)

## By year

flash redpoint 2,022 2,023 2,024 2,025 year 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 difficulty 0 5 10 15 20 25 30 35 40 45 50 Frequency → undefined (14) undefined (7) undefined (2) undefined (13) undefined (9) undefined (7) undefined (9) undefined (1) undefined (1) undefined (3) undefined (1) undefined (3) undefined (1) undefined (2) undefined (1) undefined (2) undefined (4) undefined (13) undefined (27) undefined (22) undefined (3) undefined (4) undefined (6) undefined (14) undefined (1) undefined (1) undefined (4) undefined (24) undefined (44) undefined (48) undefined (30) undefined (34) undefined (36) undefined (28) undefined (32) undefined (17) undefined (23) undefined (3) undefined (30) undefined (30) undefined (43) undefined (36) undefined (25) undefined (26) undefined (24) undefined (42) undefined (41) undefined (10) undefined (18) undefined (5) undefined (1) undefined (17) undefined (23) undefined (9) undefined (16) undefined (6) undefined (13) undefined (1) undefined (24) undefined (2) undefined (4) undefined (2) undefined (3) undefined (1) undefined (2) undefined (2) undefined (3) undefined (2) undefined (2) undefined (8) undefined (11) undefined (13) undefined (7) undefined (8) undefined (5) undefined (6) undefined (2) undefined (4) undefined (10) undefined (10) undefined (6)

## By gym

flash redpoint Klättercentret Akalla Klättercentret Solna Klättercentret Telefonplan parent_name 2,022 2,023 2,024 2,025 year 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 difficulty 0 20 40 0 20 40 0 20 40 0 20 40 Frequency → undefined (14) undefined (7) undefined (2) undefined (13) undefined (9) undefined (7) undefined (9) undefined (1) undefined (1) undefined (3) undefined (1) undefined (3) undefined (1) undefined (2) undefined (1) undefined (2) undefined (1) undefined (2) undefined (1) undefined (2) undefined (4) undefined (13) undefined (27) undefined (22) undefined (3) undefined (4) undefined (6) undefined (14) undefined (1) undefined (1) undefined (4) undefined (24) undefined (43) undefined (46) undefined (29) undefined (32) undefined (36) undefined (28) undefined (32) undefined (17) undefined (23) undefined (3) undefined (30) undefined (30) undefined (42) undefined (35) undefined (25) undefined (26) undefined (24) undefined (41) undefined (40) undefined (10) undefined (18) undefined (5) undefined (1) undefined (17) undefined (23) undefined (9) undefined (16) undefined (6) undefined (13) undefined (1) undefined (24) undefined (2) undefined (1) undefined (1) undefined (1) undefined (1) undefined (4) undefined (2) undefined (3) undefined (1) undefined (2) undefined (2) undefined (3) undefined (2) undefined (2) undefined (8) undefined (11) undefined (13) undefined (7) undefined (8) undefined (5) undefined (6) undefined (2) undefined (4) undefined (10) undefined (10) undefined (6)

## By route setter

flash redpoint Anton Cuevas Silver Breiktopf James Thanawat Benjamin Schlobohm N.N. Stina Dorwarth Sebastian Dahlgren Robert Rundin Sophie Machaczek Isabelle Fugelstad Scott Mooney Oliver Sjödin Robin Dahlberg Sofia Wideroth Tobias Lindgren Zac Chattaway route_setter 2,022 2,023 2,024 2,025 year 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 difficulty 0 5 10 15 0 5 10 15 0 5 10 15 0 5 10 15 0 5 10 15 0 5 10 15 0 5 10 15 0 5 10 15 0 5 10 15 0 5 10 15 0 5 10 15 Frequency → undefined (3) undefined (1) undefined (1) undefined (1) undefined (4) undefined (2) undefined (1) undefined (1) undefined (1) undefined (2) undefined (1) undefined (1) undefined (1) undefined (5) undefined (2) undefined (4) undefined (3) undefined (3) undefined (2) undefined (1) undefined (1) undefined (1) undefined (1) undefined (2) undefined (1) undefined (2) undefined (1) undefined (1) undefined (1) undefined (1) undefined (1) undefined (3) undefined (2) undefined (1) undefined (1) undefined (1) undefined (1) undefined (1) undefined (1) undefined (1) undefined (2) undefined (4) undefined (2) undefined (2) undefined (1) undefined (1) undefined (6) undefined (11) undefined (7) undefined (4) undefined (3) undefined (7) undefined (5) undefined (5) undefined (4) undefined (6) undefined (1) undefined (8) undefined (8) undefined (6) undefined (5) undefined (1) undefined (5) undefined (1) undefined (3) undefined (3) undefined (1) undefined (4) undefined (2) undefined (4) undefined (2) undefined (1) undefined (2) undefined (1) undefined (2) undefined (2) undefined (1) undefined (2) undefined (1) undefined (1) undefined (4) undefined (1) undefined (1) undefined (6) undefined (5) undefined (6) undefined (8) undefined (6) undefined (9) undefined (5) undefined (4) undefined (1) undefined (1) undefined (6) undefined (2) undefined (4) undefined (5) undefined (2) undefined (8) undefined (13) undefined (16) undefined (6) undefined (7) undefined (5) undefined (4) undefined (7) undefined (2) undefined (6) undefined (2) undefined (1) undefined (2) undefined (1) undefined (2) undefined (3) undefined (3) undefined (2) undefined (1) undefined (1) undefined (1) undefined (2) undefined (7) undefined (1) undefined (4) undefined (1) undefined (1) undefined (3) undefined (5) undefined (8) undefined (10) undefined (7) undefined (10) undefined (4) undefined (1) undefined (1) undefined (1) undefined (2) undefined (1) undefined (3) undefined (2) undefined (2) undefined (2) undefined (3) undefined (1) undefined (2) undefined (1) undefined (1) undefined (1) undefined (1) undefined (2) undefined (1) undefined (1) undefined (2) undefined (1) undefined (2) undefined (2) undefined (3) undefined (2) undefined (1) undefined (1) undefined (1) undefined (1) undefined (1) undefined (1) undefined (2) undefined (1) undefined (1) undefined (1) undefined (1) undefined (2) undefined (1) undefined (1) undefined (1) undefined (1) undefined (1) undefined (8) undefined (4) undefined (9) undefined (10) undefined (4) undefined (11) undefined (9) undefined (9) undefined (6) undefined (5) undefined (4) undefined (2) undefined (4) undefined (1) undefined (8) undefined (8) undefined (3) undefined (1) undefined (4) undefined (4) undefined (1) undefined (12) undefined (12) undefined (14) undefined (5) undefi

In [ ]:
await display(
  md`
## Poor man's CPR
  `,
);

Plot.plot(
  (() => {
    const data = detailedAscents
      .groupBy(pl.col('date').date.strftime('%Y-%U').alias('week'), 'type')
      .agg(
        pl.col('date').first(),
        pl.col('difficulty').min().alias('difficultyMin'),
        pl.col('difficulty').max().alias('difficultyMax'),
        pl.col('numberDifficulty').min().sub(3).alias('numberDifficultyMin'),
        pl.col('numberDifficulty').max().alias('numberDifficultyMax'),
        pl.col('date').count().alias('count'),
      );

    return {
      figure: true,
      grid: true,
      color: { legend: true, type: 'categorical', ...ascentTypeColorDomain },
      width: 1500,
      y: {
        ticks: boulderScores.filter((grade) => grade.font <= '7B').map((grade) => grade.score),
        tickFormat: (v) => boulderScores.find(({ score }) => v === score)?.font,
      },
      marks: [
        Plot.frame(),
        Plot.rectY(
          data.toRecords(),

          {
            interval: 'week',
            x: 'date',

            y1: 'numberDifficultyMin',
            y2: 'numberDifficultyMax',
            fill: 'type',
            title: (d) => {
              if (d.difficultyMin === d.difficultyMax) return d.difficultyMin;
              return `${d.difficultyMin} - ${d.difficultyMax}`;
            },
            inset: 1,
            mixBlendMode: 'color-dodge',
          },
        ),
        /*
        Plot.rectY(data.toRecords(), { interval: 'week', x: 'date', y: 'count', fill: 'gray' }),
        Plot.text(data.toRecords(), {
          x: 'date',
          y: 'count',
          text: 'count',
          lineAnchor: 'bottom',
        }),
        */
      ],
      document,
    };
  })(),
);



## Poor man's CPR
  

flash redpoint 2 2+ 3 3+ 4 4+ 5 5+ 6A 6A+ 6B 6B+ 6C 6C+ 7A 7A+ 7B Apr 2022 Jul Oct Jan 2023 Apr Jul Oct Jan 2024 Apr Jul Oct Jan 2025 2 - 6B+ 6B+ - 6C 6B+ 4+ 3 - 6B+ 3 - 6B+ 7A+ - 7B 6C 6C 6A+ - 7A 3 - 6B+ 6B - 7A+ 4+ - 6A+ 6B - 6C+ 3 - 6C 5+ - 7A 6A+ - 6B+ 6C 3 - 6B+ 3 - 6B+ 6B+ 6A+ - 6B 6B+ 3 - 6C+ 3 - 6C 6A+ - 7A 6C 3 - 6C+ 6A - 6B 6C - 7A+ 3+ - 6B 6A+ - 6B+ 5 - 6B 4+ - 6A+ 4 - 6C 6C 6C 6C 3+ - 6A+ 6C 3 - 6B+ 3 - 6C 6B+ 5+ - 7A+ 6C+ 5+ - 6A+ 4 - 6A+ 5 - 6A 6B+ - 6C 6B+ - 6C+ 6B+ - 7A 5 4 - 6A+ 3+ - 6B 3+ - 6B+ 6A+ 3+ - 6A+ 3+ - 6B 5 - 6A 6A+ - 7A 4+ - 6B+ 4+ - 6A+ 6A+ - 6C 3 - 6A+ 6A - 6B+ 5+ - 6C+ 6A+ - 7A 6B+ - 7B 6B+ - 7A+ 5 - 6C 7A - 7A+ 3 - 5 4 - 6C 3+ - 6A+ 6A+ - 7A+ 6B+ - 7A+ 4+ - 6B 3+ - 6B+ 6A+ - 6B+ 6A+ - 6B+ 6B+ - 6C 3 - 6C 3+ - 6A+ 3 - 6B+ 5 - 6B+ 6B 6C - 7A 6C+ - 7B 6B 3+ - 6B+ 6A+ 3 - 6A+ 3 - 6B+ 6B - 6C+ 6C 6B+ - 6C 3 - 6B+ 6B - 6C+ 6A+ - 6B 6B - 6C 3 - 6A 3 - 6A 6B 6C 3+ - 6B 3 - 6B 3 - 6C 3 - 5+ 3 - 6B+ 6B+ - 6C 4+ 3+ - 6B+ 3+ - 6B 6A - 7A 5+ - 6A 6B+ - 7A 6A - 6C 6C 6B - 6C 6A - 6C 6A+ - 7A+ 6B - 6C+ 6C - 7B 3+ - 6B+ 3 - 6A 3+ - 6B+ 4+ - 6B+ 3 - 6B+ 7A+ 6C 4+ - 6B 3+ - 6C 3 - 6A+ 5+ - 6B 6B+ - 6C 6C - 7A 6B 3+ - 7A 6B+ - 7A+ 6A+ - 7A+ 3 - 6A+ 3 - 6B+ 4+ - 6C+ 3 - 6B+ 6C 3 - 6A+ 4 - 6A+ 3 - 6C 3 - 6B 6B+ - 6C+ 3+ - 6A+ 4 - 6A+ 6B+ - 6C+ 6B+ - 6C+ 6C 3 - 6C 6C - 7A 3 - 6A+ 4+ - 6B 3+ - 6B+ 4+ 5 - 6C 3+ - 6B+ 6B+ 4 - 6A+ 6A - 6A+ 3+ - 6C 4 - 6C+ 7A 6B+ - 7A+ 3+ - 6B 6A+ - 7A 3+ - 6A+ 3+ - 6B 6C+ 3 - 6B+ 6A 5+ - 6C 6B - 6C 6C+ - 7A 3 - 5+ 6B 6B - 7A+ 7A - 7A+ 4 - 6B 6A+ - 6B 6B - 6C 3 - 6B 6B+ - 7A 6B+ 3+ - 6B+

In [ ]:
Plot.plot(
  (() => {
    const data = detailedAscents
      .groupBy(pl.col('date').date.strftime('%Y-%U').alias('grouped'))
      .agg(
        pl.col('date').first().cast(pl.Date).alias('date'),
        pl.col('numberDifficulty').sum().alias('totalNumberDifficulty'),
        pl.col('date').count().alias('count'),
        pl.col('difficulty'),
      )
      .toRecords();
    return {
      figure: true,
      color: { scheme: 'YlGn', domain: [0, 3000] },
      width: 1500,
      marks: [
        Plot.barX(data, {
          interval: 'week',
          x: 'date',
          fill: 'totalNumberDifficulty',
          title: (d) =>
            Object.entries(
              d.difficulty.reduce((acc, grade) => {
                if (!acc[grade]) {
                  acc[grade] = 0;
                }
                acc[grade] += 1;
                return acc;
              }, {}),
            )
              .toSorted((a, b) => a[0].localeCompare(b[0]))
              .reduce((acc, ascents) => [...acc, `${ascents[0]} x${ascents[1]}`], [] as string[])
              .join('; '),
        }),
      ],
      document,
    };
  })(),
);


Apr 2022 Jul Oct Jan 2023 Apr Jul Oct Jan 2024 Apr Jul Oct Jan 2025 3 x1; 3+ x1; 4 x1; 4+ x2; 5 x2; 5+ x1; 6A x2; 6A+ x2; 6B x1 5 x1; 5+ x2; 6A x2; 6C x1 3 x1; 3+ x2; 4 x2; 4+ x3; 5 x1; 5+ x1; 6A x2; 6B x2; 6B+ x3; 6C x3 3+ x1; 4 x2; 4+ x1; 5+ x1; 6A x1; 6A+ x1; 6C x2 6A x1; 6A+ x1; 6B x1; 6B+ x1; 6C x1 5 x2; 5+ x2; 6A x3; 6A+ x3; 6B x3; 6B+ x2; 6C x1 3+ x1; 5+ x1; 6B x1; 6B+ x1; 7A+ x1 3 x2; 3+ x1; 4 x2; 4+ x2; 5 x2; 6A x3; 6A+ x2; 6B x1; 6B+ x2; 6C+ x1 3 x2; 3+ x2; 4 x2; 5+ x2; 6A x4; 6A+ x2; 6B x1; 6B+ x1 4 x1; 6A x1; 6A+ x2; 6B+ x2; 6C x2; 6C+ x1; 7A x1 3+ x2; 4 x2; 5 x1; 5+ x2; 6A x1; 6A+ x5; 6B x6; 6B+ x1; 6C x3; 6C+ x1; 7A x1; 7A+ x1 4 x2; 4+ x1; 5 x1; 5+ x4; 6A x2; 6A+ x1 6B x1; 6B+ x1; 6C x1; 6C+ x1; 7A x1 3 x1; 3+ x1; 4+ x2; 5 x1; 6B+ x3; 6C x1 3 x1; 4 x1; 4+ x2; 5+ x2; 6A x1; 6A+ x2; 6B x2; 6B+ x1; 6C x2 3+ x1; 4 x1; 4+ x1; 5+ x2; 6A+ x1; 6B x2; 6B+ x1; 7A x1 6A x1; 6A+ x1; 6C x1 3+ x1; 6B x1; 6B+ x1 3+ x2; 4 x1; 5 x1; 6B x2; 6B+ x1; 6C+ x1; 7A x1 3 x1; 3+ x1; 4+ x1; 5 x1; 5+ x1; 6A+ x1 4+ x3; 5+ x3; 6A x1; 6A+ x1; 6B x2; 6B+ x2; 6C x1 3 x1; 4 x1; 4+ x1; 5 x4; 5+ x2; 6A x3; 6A+ x2; 6B x2; 6B+ x1; 6C x2; 6C+ x3; 7A+ x1 4+ x2; 6A x3; 6A+ x1; 6B x2; 6C x1; 6C+ x1 3+ x1; 4 x1; 4+ x1; 5 x2; 5+ x2; 6A x1; 6A+ x1; 6B+ x1; 6C+ x1 3+ x1; 4+ x2; 6A x3; 6A+ x1 3 x1; 4 x1; 5 x6; 6A x1; 6A+ x2; 6B x3; 6B+ x1; 6C x2; 6C+ x1 3+ x1; 4 x1; 4+ x1; 5 x1; 5+ x2; 6A x3; 6A+ x3; 6B+ x3; 7A+ x1; 7B x1 3 x1; 3+ x2; 4 x1; 4+ x2; 5 x1; 5+ x4; 6A x2; 6A+ x4; 6B x4; 6B+ x4; 6C x1 3 x1; 3+ x2; 4+ x2; 5 x3; 5+ x1; 6A x3; 6A+ x2; 6B x2; 6B+ x1; 6C x2 5+ x1; 6A x2; 6B+ x1; 6C x1; 6C+ x1 3 x1; 4 x1; 4+ x2; 5+ x1; 6A x2; 6A+ x1; 6C x1 3 x2; 3+ x2; 4 x3; 4+ x1; 5 x3; 5+ x2; 6A x1; 6A+ x1; 6B x2; 6B+ x1; 7A+ x1 3+ x1; 4+ x1; 5 x1; 5+ x1; 6A x1; 6B+ x1; 6C+ x1 4+ x1; 5 x1; 5+ x3; 6A x2; 6A+ x1 3 x1; 3+ x1; 4+ x1; 5 x2; 5+ x1; 6A x4; 6A+ x3; 6B x4; 6B+ x1; 6C x2 3+ x1; 4+ x1; 5+ x1; 6A x1; 6A+ x2; 6B+ x1 4 x1; 6A+ x1; 6B+ x2; 6C x1 3 x2; 3+ x1; 4 x2; 4+ x4; 5+ x4; 6A x4; 6A+ x3; 6C x1; 7A x2; 7A+ x1 6A+ x1; 6B x1 3 x2; 3+ x1; 4 x1; 4+ x2; 5 x1; 6A x2; 6A+ x1; 6B x1; 6B+ x2; 6C+ x1; 7A+ x1 3 x1; 3+ x1; 4 x3; 4+ x1; 5 x3; 6A x2; 6A+ x3; 6B x1; 6B+ x2; 6C x2; 6C+ x1; 7A x1 3+ x1; 4 x1; 4+ x1; 6A x1; 6A+ x2; 6B x2; 6B+ x1; 6C+ x1 3+ x1; 4+ x1; 5 x1; 5+ x3; 6A x2; 6A+ x2; 6B x1; 6B+ x3; 6C x3 4+ x1; 6A+ x1 2 x3; 3 x1; 3+ x1; 4 x2; 4+ x1; 5+ x1; 6A+ x2; 6B x1; 6B+ x1 5+ x1; 6A x1; 6A+ x2; 6B x2; 6B+ x2; 6C x2; 6C+ x1; 7A x1 3+ x2; 4 x1; 4+ x1; 5+ x1; 6A x1; 6B x1; 6B+ x3; 6C+ x1 3+ x1; 4 x1; 6A+ x2; 6B+ x1 3 x1; 4 x2; 5 x2; 6A x1; 6A+ x2; 6B x1; 6B+ x1; 6C x1; 6C+ x1; 7A x1 3 x1; 3+ x2; 4 x2; 4+ x1; 5 x2; 6A x1; 6A+ x1; 6B x1; 6B+ x2; 6C x2 3 x2; 4 x1; 5 x4; 5+ x1; 6A x3; 6A+ x3; 6B x2; 6B+ x1; 6C+ x3; 7A x1 3 x1; 5 x2; 6A x1; 6B x1; 6B+ x1; 6C x1; 6C+ x1 6A x1; 6B x1; 6B+ x1 3+ x1; 4 x3; 4+ x3; 5 x2; 5+ x2; 6A x3; 6A+ x1; 6B x1; 6B+ x1; 6C x2; 7A+ x1 4+ x1; 5+ x2; 6A+ x2; 6B x1; 6B+ x1 3 x1; 4+ x2; 5+ x1; 6A x1; 6A+ x2; 6B x1; 6C x1 5+ x1; 6A+ x1; 7A x1; 7A+ x1 6A+ x1; 6B x1; 6B+ x1 4+ x1; 7A+ x1 4+ x1; 6A+ x1; 6B+ x2; 6C x1; 6C+ x1 4 x1; 4+ x1; 6A x1; 6B x1; 6B+ x3; 6C x2; 6C+ x1; 7B x1 3 x1; 3+ x2; 4+ x3; 5 x1; 5+ x2; 6A x1; 6A+ x1; 6B x1; 6B+ x1; 6C+ x2; 7A x1; 7B x1 3 x1; 4 x1; 4+ x1; 5 x1; 6A x1; 6A+ x1; 6B+ x1; 7A x1 6A+ x1; 6B x1 5 x1 3+ x1; 4+ x1; 5 x1; 5+ x1; 6A x3; 6A+ x1; 6B+ x2; 6C x1; 6C+ x1; 7A x3 3 x1; 4 x2; 4+ x3; 5 x1; 5+ x2 3 x1; 3+ x1; 4 x1; 4+ x1; 5 x1; 6A x1; 6A+ x3; 6B x1; 6B+ x2; 6C+ x1; 7A x1 3 x1; 4 x1; 4+ x1; 5+ x1; 6A x1; 6B x1 6C x2 3+ x1; 4 x2; 4+ x1; 5+ x1; 6A x1; 6A+ x1; 6B+ x2; 6C x1 6C x1 4 x1; 5 x1; 6A x1; 6B+ x1; 6C x1 4 x1; 5 x2; 6A x1; 6B x1; 6B+ x1; 6C x2; 6C+ x1 3 x2; 3+ x1; 4 x1; 5 x1; 6A x1; 6A+ x1; 6B x1; 6B+ x2; 6C+ x2; 7A x2 3 x2; 3+ x2; 4 x1; 4+ x2; 5 x2; 5+ x5; 6A x2; 6A+ x1; 6B x2; 6B+ x1; 6C x1 3 x2; 4 x1; 4+ x2; 5+ x1; 6A x1; 6A+ x1; 6C x1 4+ x1; 5 x3; 6A x1; 6A+ x1; 6B x3; 6C x1; 7B x1 3+ x1; 4+ x1; 5+ x1; 6A+ x2; 6C x1 3 x1; 4+ x1; 5+ x1; 6A x2; 6A+ x2; 6B x1; 6B+ x2; 6C x3; 6C+ x1 4+ x1; 5 x3; 5+ x1; 6A x1;

In [ ]:
import * as d3 from 'npm:d3';

Plot.plot(
  (() => {
    const data = detailedAscents
      .groupBy(pl.col('date').date.strftime('%Y-%U').alias('grouped'))
      .agg(
        pl.col('date').first().date.strftime('%Y-%m-%d'),
        pl.col('numberDifficulty').sum().alias('totalNumberDifficulty'),
        pl.col('date').count().alias('count'),
        pl.col('difficulty'),
      )
      .toRecords();
    return {
      figure: true,
      x: { tickFormat: (d) => `Week ${d + 1}`, tickRotate: 90 },
      y: { tickFormat: '', tickSize: 0 },
      color: { scheme: 'YlGn', domain: [0, 3000] },
      marginBottom: 47,
      marks: [
        Plot.cell(data, {
          x: (d) => d3.utcWeek.count(d3.utcYear(new Date(d.date)), new Date(d.date)),
          y: (d) => new Date(d.date).getUTCFullYear(),
          fill: 'totalNumberDifficulty',
          title: (d) =>
            Object.entries(
              d.difficulty.reduce((acc, grade) => {
                if (!acc[grade]) {
                  acc[grade] = 0;
                }
                acc[grade] += 1;
                return acc;
              }, {}),
            )
              .toSorted((a, b) => a[0].localeCompare(b[0]))
              .reduce((acc, ascents) => [...acc, `${ascents[0]} x${ascents[1]}`], [] as string[])
              .join('; '),
          inset: 1,
        }),
      ],
      document,
    };
  })(),
);


2022 2023 2024 2025 Week 1 Week 2 Week 3 Week 4 Week 5 Week 6 Week 7 Week 8 Week 9 Week 10 Week 11 Week 12 Week 13 Week 14 Week 15 Week 16 Week 17 Week 18 Week 19 Week 20 Week 21 Week 22 Week 23 Week 24 Week 25 Week 26 Week 27 Week 28 Week 29 Week 30 Week 31 Week 32 Week 33 Week 34 Week 35 Week 36 Week 37 Week 38 Week 39 Week 40 Week 41 Week 42 Week 43 Week 44 Week 45 Week 46 Week 47 Week 48 Week 49 Week 50 Week 51 Week 52 3 x1; 3+ x2; 4 x2; 4+ x3; 5 x1; 5+ x1; 6A x2; 6B x2; 6B+ x3; 6C x3 3+ x1; 4 x1; 4+ x1; 5+ x2; 6A+ x1; 6B x2; 6B+ x1; 7A x1 6A x1; 6A+ x1; 6B x1; 6B+ x1; 6C x1 3 x1; 3+ x2; 4+ x2; 5 x3; 5+ x1; 6A x3; 6A+ x2; 6B x2; 6B+ x1; 6C x2 4 x1; 6A x1; 6A+ x2; 6B+ x2; 6C x2; 6C+ x1; 7A x1 3 x2; 3+ x1; 4 x1; 4+ x2; 5 x1; 6A x2; 6A+ x1; 6B x1; 6B+ x2; 6C+ x1; 7A+ x1 3+ x1; 4 x1; 5+ x1; 6A+ x1; 6B+ x1; 7A+ x1 3+ x1; 6B x1; 6B+ x1 3+ x1; 4+ x1; 5 x1; 5+ x3; 6A x2; 6A+ x2; 6B x1; 6B+ x3; 6C x3 3 x1; 3+ x1; 4+ x2; 5 x1; 6B+ x3; 6C x1 3 x2; 3+ x1; 4 x2; 4+ x4; 5+ x4; 6A x4; 6A+ x3; 6C x1; 7A x2; 7A+ x1 3 x2; 3+ x1; 4 x2; 4+ x2; 5 x2; 6A x3; 6A+ x2; 6B x1; 6B+ x2; 6C+ x1 3 x1; 3+ x1; 4+ x1; 5 x2; 5+ x1; 6A x4; 6A+ x3; 6B x4; 6B+ x1; 6C x2 5+ x1; 6A+ x1; 7A x1; 7A+ x1 3 x1; 5 x2; 6A x1; 6B x1; 6B+ x1; 6C x1; 6C+ x1 3+ x1; 5+ x1; 6B x1; 6B+ x1; 7A+ x1 4 x1; 5 x2; 6A x1; 6B x1; 6B+ x1; 6C x2; 6C+ x1 3 x1; 4+ x1; 5+ x1; 6A x2; 6A+ x2; 6B x1; 6B+ x2; 6C x3; 6C+ x1 3 x1; 3+ x2; 4 x2; 4+ x1; 5 x2; 6A x1; 6A+ x1; 6B x1; 6B+ x2; 6C x2 6A+ x1; 6B x1 5 x1; 5+ x1; 6A x1; 6A+ x2; 6B x1; 6C x2 3+ x1; 4 x3; 4+ x3; 5 x2; 5+ x2; 6A x3; 6A+ x1; 6B x1; 6B+ x1; 6C x2; 7A+ x1 4 x1; 5+ x1; 6B x1; 6B+ x1; 6C+ x1; 7A x1 4 x2; 4+ x1; 5 x1; 5+ x4; 6A x2; 6A+ x1 4 x1; 4+ x3; 5+ x1; 6A x2; 6A+ x1 4+ x1; 7A+ x1 3+ x1; 4+ x1; 5 x1; 5+ x1; 6A x1; 6B+ x1; 6C+ x1 4 x2; 5+ x1; 6A x1; 6A+ x1 3+ x1; 4+ x1; 5+ x1; 6A x1; 6A+ x2; 6B+ x1 3 x1; 4 x1; 4+ x2; 5+ x1; 6A x2; 6A+ x1; 6C x1 3 x2; 4+ x1; 5 x1; 5+ x2; 6A x1; 6A+ x3; 6B x3 6B+ x1 6A x1; 6A+ x1; 6C x1 5 x2; 5+ x2; 6A x3; 6A+ x3; 6B x3; 6B+ x2; 6C x1 3+ x1; 4 x1; 4+ x1; 5 x1; 5+ x2; 6A x3; 6A+ x3; 6B+ x3; 7A+ x1; 7B x1 3 x1; 3+ x1; 4+ x1; 5 x1; 5+ x1; 6A+ x1 4+ x2; 5+ x1; 6A x1; 6A+ x1; 6B+ x1; 6C x1; 7A x1 3+ x1; 4 x2; 4+ x1; 5+ x1; 6A x1; 6A+ x1; 6C x2 3 x1; 3+ x2; 4 x1; 4+ x2; 5 x1; 5+ x4; 6A x2; 6A+ x4; 6B x4; 6B+ x4; 6C x1 3+ x2; 4 x2; 5 x1; 5+ x2; 6A x1; 6A+ x5; 6B x6; 6B+ x1; 6C x3; 6C+ x1; 7A x1; 7A+ x1 3 x1; 3+ x1; 4 x3; 4+ x1; 5 x3; 6A x2; 6A+ x3; 6B x1; 6B+ x2; 6C x2; 6C+ x1; 7A x1 3 x1; 4 x2; 4+ x1; 5 x2; 5+ x1; 6A x1 3+ x1; 4 x1; 4+ x1; 5+ x1; 6A+ x2; 6B+ x1; 6C x1 3+ x1; 4 x1; 4+ x1; 6A x1; 6A+ x2; 6B x2; 6B+ x1; 6C+ x1 3 x1; 4 x1; 4+ x1; 5+ x1; 6A x1; 6B x1 4 x1; 6A+ x1; 6B+ x2; 6C x1 3+ x1; 4+ x2; 6A+ x1; 6B x1; 7A x1 6A+ x1; 6C x1; 7A+ x1 3+ x1; 4 x1; 5+ x2; 6A+ x1; 6B x2 3 x2; 3+ x1; 4 x1; 5 x1; 6A x1; 6A+ x1; 6B x1; 6B+ x2; 6C+ x2; 7A x2 3 x2; 3+ x2; 4 x1; 4+ x2; 5 x2; 5+ x5; 6A x2; 6A+ x1; 6B x2; 6B+ x1; 6C x1 3 x1; 4 x1; 4+ x1; 5 x1; 6A x1; 6A+ x1; 6B+ x1; 7A x1 4+ x1; 6A+ x1; 6B+ x2; 6C x1; 6C+ x1 3 x1; 3+ x3; 4 x3; 5 x1; 6A x1; 6B x1; 6B+ x2 3 x1; 4 x2; 4+ x3; 5 x1; 5+ x2 5+ x1; 6A x1; 6A+ x2; 6B x2; 6B+ x2; 6C x2; 6C+ x1; 7A x1 6B x1; 6B+ x1; 6C x1; 6C+ x1; 7A x1 3 x1; 3+ x1; 4 x2; 4+ x1; 5 x1; 5+ x2; 6A x2 4+ x1; 5 x3; 6A x1; 6A+ x1; 6B x3; 6C x1; 7B x1 6A+ x1; 6B x1; 6B+ x1 6A+ x1; 6B x1 3 x2; 3+ x2; 4 x3; 4+ x1; 5 x3; 5+ x2; 6A x1; 6A+ x1; 6B x2; 6B+ x1; 7A+ x1 3 x1; 3+ x1; 4 x1; 4+ x1; 5 x1; 6A x1; 6A+ x3; 6B x1; 6B+ x2; 6C+ x1; 7A x1 2 x3; 3 x1; 3+ x1; 4 x2; 4+ x1; 5+ x1; 6A+ x2; 6B x1; 6B+ x1 3+ x1; 4 x1; 6A+ x2; 6B+ x1 6A x1; 6A+ x1 6C x1 3 x1; 4 x1; 5 x6; 6A x1; 6A+ x2; 6B x3; 6B+ x1; 6C x2; 6C+ x1 4+ x1; 5+ x1; 6A x1; 6B+ x1; 6C x1 4+ x1; 6A+ x1 5 x1; 5+ x2; 6A x2; 6C x1 3+ x1; 4 x1; 5 x1; 5+ x1; 6A x2; 6A+ x2; 6B x2; 6B+ x1 3 x1; 4+ x1; 6A x1; 6B x1; 6C x1; 7A x1; 7A+ x1 4+ x1; 5 x1; 5+ x3; 6A x2; 6A+ x1 5 x1 6C x2 3+ x1; 5 x1; 5+ x1; 6A x1; 6A+ x1; 6B x1; 6C x1; 7A+ x1 3 x1; 3+ x2; 4+ x3; 5 x1; 5+ x2; 6A x1; 6A+ x1; 6B x1; 6B+ x1; 6C+ x2; 7A x1; 7B x1 3 x2; 4 x1; 5 x4; 5+ x1; 